In [1]:
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy, get_error_measures, collect_stats, print_average_stats
from src.models import SimpleANN, SynapticSNN, LeakySNN, DoubleLeakySNN
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Data

In [3]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [4]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.75 * len(all_data)), round(0.15 * len(all_data)), round(0.1 * len(all_data))))

In [5]:
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

### Simple regressor

In [6]:
model1 = SimpleANN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model1)

TwoLayerPerceptron(
  (lin1): Linear(in_features=8, out_features=3500, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=3500, out_features=1, bias=True)
)


In [7]:
training_loop(model1, train_loader, valid_loader, device, num_epochs=20, lr=0.003, validation=True)

  5%|▌         | 1/20 [00:02<00:45,  2.39s/it]

MAE: 6.977179198733644, MSE:100.34206957404284,
RMSE: 10.017088877215917, IA: 0.8486668125695818, MAPE: 44.371183599082215%


 10%|█         | 2/20 [00:02<00:21,  1.20s/it]

MAE: 6.859676299894476, MSE:97.8839128699343,
RMSE: 9.893629913734104, IA: 0.8635434747844735, MAPE: 44.288341437970004%


 15%|█▌        | 3/20 [00:03<00:14,  1.21it/s]

...

MAE: 6.551630107722531, MSE:88.51039284960504,
RMSE: 9.407996218621957, IA: 0.8712973891329541, MAPE: 41.5764665007009%


 95%|█████████▌| 19/20 [00:09<00:00,  2.57it/s]

MAE: 6.593156081817054, MSE:88.8215455622859,
RMSE: 9.424518319908232, IA: 0.8761724603181775, MAPE: 43.051662889198596%


100%|██████████| 20/20 [00:09<00:00,  2.11it/s]

MAE: 6.9960809159141055, MSE:97.06604644953235,
RMSE: 9.852210231695848, IA: 0.8679841691513017, MAPE: 48.403454706458916%
Accuracy on validation dataset: 47.980000000000004%


In [8]:
print(f'Accuracy on test set: {get_accuracy(model1, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model1, test_loader, device=device, print_e=True)

Accuracy on test set: 47.53%
MAE: 6.584075526390311, MSE:81.99053064639483,
RMSE: 9.054862265456876, IA: 0.8827876434020421, MAPE: 46.74369985409349%


(6.584075526390311,
 81.99053064639483,
 9.054862265456876,
 0.8827876434020421,
 46.74369985409349)

### SNN with Leaky layer

In [9]:
model2 = LeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1, num_steps=25)
print(model2)

LeakySNN(
  (fc1): Linear(in_features=8, out_features=3500, bias=True)
  (lif): Leaky()
  (fc2): Linear(in_features=3500, out_features=1, bias=True)
)


In [10]:
training_loop(model2, train_loader, valid_loader, device, num_epochs=25, lr=0.001, validation=True)

  4%|▍         | 1/25 [00:04<01:59,  4.97s/it]

MAE: 6.771930640556909, MSE:97.31362862957145,
RMSE: 9.86476703372013, IA: 0.8662075685537425, MAPE: 42.58734285394422%


  8%|▊         | 2/25 [00:09<01:52,  4.91s/it]

MAE: 6.901768796016715, MSE:98.0451238094351,
RMSE: 9.901773770867273, IA: 0.8591297149681488, MAPE: 39.89053553427123%


 12%|█▏        | 3/25 [00:14<01:48,  4.92s/it]

...

MAE: 6.11515609935529, MSE:73.42830192414766,
RMSE: 8.569031562793294, IA: 0.9051800888525983, MAPE: 40.48702337767838%


 96%|█████████▌| 24/25 [02:00<00:04,  4.98s/it]

MAE: 6.042265920280721, MSE:72.06809979207522,
RMSE: 8.489293244556652, IA: 0.9079222372639418, MAPE: 34.68317112339825%


100%|██████████| 25/25 [02:05<00:00,  5.03s/it]

MAE: 6.309930639976711, MSE:79.58138106118966,
RMSE: 8.920839706058485, IA: 0.894345374289621, MAPE: 35.43424808735098%
Accuracy on validation dataset: 49.830000000000005%


In [11]:
print(f'Accuracy on test set: {get_accuracy(model2, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model2, test_loader, device=device, print_e=True)

Accuracy on test set: 51.160000000000004%
MAE: 5.972411362158683, MSE:73.31110708098069,
RMSE: 8.562190553881681, IA: 0.8962826474424391, MAPE: 34.6520643229448%


(5.972411362158683,
 73.31110708098069,
 8.562190553881681,
 0.8962826474424391,
 34.6520643229448)

### SNN with Synaptic layer

In [12]:
model3 = SynapticSNN(num_inputs=8, num_hidden=3500, num_outputs=1, num_steps=50)
print(model3)

SynapticSNN(
  (fc1): Linear(in_features=8, out_features=3500, bias=True)
  (lif): Synaptic()
  (fc2): Linear(in_features=3500, out_features=1, bias=True)
)


In [13]:
training_loop(model3, train_loader, valid_loader, device, num_epochs=25, lr=0.0005, validation=True)

  4%|▍         | 1/25 [00:10<04:14, 10.62s/it]

MAE: 6.909225411635603, MSE:95.00316251195699,
RMSE: 9.74695657689912, IA: 0.8730246451840118, MAPE: 46.58436028497872%


  8%|▊         | 2/25 [00:20<03:58, 10.37s/it]

MAE: 6.61289022313377, MSE:86.37375911138491,
RMSE: 9.293748388641953, IA: 0.891136062933712, MAPE: 45.24222761881174%


 12%|█▏        | 3/25 [00:31<03:50, 10.49s/it]

...

MAE: 5.914231181282529, MSE:65.17750602677856,
RMSE: 8.073258699359172, IA: 0.9218099389258596, MAPE: 34.949823254561196%


 96%|█████████▌| 24/25 [04:09<00:10, 10.40s/it]

MAE: 6.90268213294145, MSE:80.44699312128886,
RMSE: 8.969224778167222, IA: 0.9081367779385366, MAPE: 49.10808721514611%


100%|██████████| 25/25 [04:19<00:00, 10.38s/it]

MAE: 5.509052003015673, MSE:55.67434434941341,
RMSE: 7.461524264479303, IA: 0.9316703376768273, MAPE: 33.72133850987771%
Accuracy on validation dataset: 52.14%


In [14]:
print(f'Accuracy on test set: {get_accuracy(model3, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model3, test_loader, device=device, print_e=True)

Accuracy on test set: 53.0%
MAE: 5.332241890380857, MSE:53.110854451668686,
RMSE: 7.287719427342733, IA: 0.9303150895932372, MAPE: 34.26196541916594%


(5.332241890380857,
 53.110854451668686,
 7.287719427342733,
 0.9303150895932372,
 34.26196541916594)

### SNN with two Leaky layers

In [15]:
model4 = DoubleLeakySNN(num_inputs=8, num_hidden=3500, num_outputs=1)
print(model4)

DoubleLeakySNN(
  (lin1): Linear(in_features=8, out_features=3500, bias=True)
  (lif1): Leaky()
  (lif2): Leaky()
  (lin2): Linear(in_features=3500, out_features=1, bias=True)
)


In [16]:
training_loop(model4, train_loader, valid_loader, device, num_epochs=25, lr=0.002, validation=True)

  4%|▍         | 1/25 [00:08<03:18,  8.27s/it]

MAE: 8.698820817539458, MSE:138.23902871559272,
RMSE: 11.757509460578492, IA: 0.7005968754790048, MAPE: 60.63466696968963%


  8%|▊         | 2/25 [00:16<03:11,  8.31s/it]

MAE: 7.131606938521986, MSE:102.52068320709769,
RMSE: 10.125249784923712, IA: 0.8469535881528957, MAPE: 45.17123081718359%


 12%|█▏        | 3/25 [00:25<03:08,  8.55s/it]

...

MAE: 6.878647420034243, MSE:97.08553656817278,
RMSE: 9.853199306223983, IA: 0.8607174308468796, MAPE: 45.34781950406298%


 96%|█████████▌| 24/25 [03:26<00:08,  8.66s/it]

MAE: 6.911283267577948, MSE:97.30215012302486,
RMSE: 9.864185223475118, IA: 0.8578450871803953, MAPE: 45.27262108124026%


100%|██████████| 25/25 [03:34<00:00,  8.57s/it]

MAE: 6.9349132863083325, MSE:99.96194535219932,
RMSE: 9.998097086555987, IA: 0.8525111392066496, MAPE: 42.31609397360791%
Accuracy on validation dataset: 47.28%


In [17]:
print(f'Accuracy on test set: {get_accuracy(model4, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model4, test_loader, device=device, print_e=True)

Accuracy on test set: 48.0%
MAE: 6.554738069168408, MSE:84.30115118115977,
RMSE: 9.181565834930323, IA: 0.8682392415719199, MAPE: 39.55185705235553%


(6.554738069168408,
 84.30115118115977,
 9.181565834930323,
 0.8682392415719199,
 39.55185705235553)

### Repeated training on random datasets

In [9]:
err_model1 = [0., 0., 0., 0., 0., 0.]
err_model2 = [0., 0., 0., 0., 0., 0.]
err_model3 = [0., 0., 0., 0., 0., 0.]
err_model4 = [0., 0., 0., 0., 0., 0.]

In [10]:
trials = 5
for i in range(trials):
    # Prepare data loaders.
    train_dataset, test_dataset = torch.utils.data.random_split(all_data, (round(0.8 * len(all_data)), round(0.2 * len(all_data))))
    train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
    test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)

    # Prepare models.
    model1 = SimpleANN(num_inputs=8, num_hidden=4000, num_outputs=1)
    model2 = LeakySNN(num_inputs=8, num_hidden=4000, num_outputs=1, num_steps=25)
    model3 = SynapticSNN(num_inputs=8, num_hidden=4000, num_outputs=1, num_steps=50)
    model4 = DoubleLeakySNN(num_inputs=8, num_hidden=4000, num_outputs=1, num_steps=20)

    # Train models.
    print(f'------------------------------------\nTraining {i+1}')
    training_loop(model1, train_loader, test_loader, device, num_epochs=25, lr=0.005, validation=False)
    training_loop(model2, train_loader, test_loader, device, num_epochs=25, lr=0.001, validation=False)
    training_loop(model3, train_loader, test_loader, device, num_epochs=25, lr=0.0005, validation=False)
    training_loop(model4, train_loader, test_loader, device, num_epochs=25, lr=0.001, validation=False)

    # Gather results.
    collect_stats(model1, err_model1, test_loader, device)
    collect_stats(model2, err_model2, test_loader, device)
    collect_stats(model3, err_model3, test_loader, device)
    collect_stats(model4, err_model4, test_loader, device)

------------------------------------
Training 1


100%|██████████| 25/25 [00:11<00:00,  2.21it/s]


Accuracy on validation dataset: 48.79%


100%|██████████| 25/25 [02:15<00:00,  5.43s/it]


Accuracy on validation dataset: 50.690000000000005%


...

Accuracy on validation dataset: 54.1%


100%|██████████| 25/25 [04:40<00:00, 11.20s/it]


Accuracy on validation dataset: 43.53%


100%|██████████| 25/25 [03:30<00:00,  8.43s/it]


Accuracy on validation dataset: 54.1%


In [11]:
# Print average results
print('------Simple ANN ------')
print_average_stats(err_model1, trials)
print('------LIF SNN ------')
print_average_stats(err_model2, trials)
print('------Synaptic SNN ------')
print_average_stats(err_model3, trials)
print('------Double LIF SNN ------')
print_average_stats(err_model4, trials)

------Simple ANN ------
MAE: 6.208275137208618, MSE:80.33695688073658,
RMSE: 8.96052686643614, IA: 0.8727338962870699,
MAPE: 37.073615240599054 %, acc: 49.446 %

------LIF SNN ------
MAE: 5.84460810140206, MSE:66.32328005726299,
RMSE: 8.138005750333555, IA: 0.9009785371074507,
MAPE: 38.07243964931761 %, acc: 51.422000000000004 %

------Synaptic SNN ------
MAE: 5.832383813015213, MSE:61.60718308915766,
RMSE: 7.840439236958607, IA: 0.9151266930054927,
MAPE: 38.96341075980955 %, acc: 49.955999999999996 %

------Double LIF SNN ------
MAE: 5.750469359445434, MSE:66.74097353878959,
RMSE: 8.16253254611893, IA: 0.9000361252886762,
MAPE: 37.34842986271641 %, acc: 53.212 %

